<a href="https://colab.research.google.com/github/anuga-community/anuga-clinic/blob/master/notebooks/notebook5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANUGA Clinic

## Notebook 5: Monai Valley Tsunami runup 

Validation of the AnuGA implementation of the shallow water wave equation.
This script sets up Okushiri Island benchmark as published at the

The Third International Workshop on Long-Wave Runup Models
June 17-18 2004

Wrigley Marine Science Center
Catalina Island, California
http://www.cee.cornell.edu/longwave/


The validation data was downloaded and made available in this directory
for convenience but the original data is available at
http://www.cee.cornell.edu/longwave/index.cfm?page=benchmark&problem=2
where a detailed description of the problem is also available.


### Installation

These notebooks have been designed to run in the google `colaboratory` environment, which provides a jupyter notebook environment running on a virtual machine on the cloud. To use this environment you need a google account so that your notebook can be saved on google drive. 

If on github, first follow the link `View in Colaboratory' to start running on google's colab environment. 

Then run the following cell to install the dependencies and some extra code for visualising on Colaboratory.

Wait until you see the comment *(4) Ready to go* before proceeding to subsequent commands. 

The install should take less than a minute (and quicker if you have already run this earlier).

In [ ]:
import os
import sys

try:
  # On colab we can install all the packages we need via the notebook
  #
  # First download the clinic repository

  HOME = '/content'
  import os
  os.chdir(HOME)
  !git clone https://github.com/anuga-community/anuga-clinic.git

  # Now install environment using tool
  !/bin/bash /content/anuga-clinic/anuga_tools/install_anuga_colab.sh
 
except:
  from pathlib import Path
  HOME = str(Path.home())


### Import ANUGA and Tools

The `anuga_clinic` package has been installed, so we can now `import` our standard libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

import anuga

## The Code 

This code is taken from `anuga_core/validation_tests/experimental_data/okishuri/run_okishuri.py`.

First we define the location of our data files, which specify:

   (1) the incoming tsunami wave, 
   
   (2) the bathymetry file, 
   
   (3) the measured stage height at 3 gauge locations.


In [ ]:
# Incoming boundary wave
boundary_filename = os.path.join(HOME, 'anuga-clinic/data/Okushiri/Benchmark_2_input.txt')

# Digital Elevation Model
bathymetry_filename = os.path.join(HOME, 'anuga-clinic/data/Okushiri/Benchmark_2_Bathymetry.xya')

# Observed timeseries
gauge_filename = os.path.join(HOME, 'anuga-clinic/data/Okushiri/output_ch5-7-9.txt')



### Load Barthymetry Data

Using in the barthymetry data provided from the workshop. We need to reshape the data and form a raster (x,y,Z).

In [ ]:
xya = np.loadtxt(bathymetry_filename, skiprows=1, delimiter=',')
  
X = xya[:,0].reshape(393,244)
Y = xya[:,1].reshape(393,244)
Z = xya[:,2].reshape(393,244)


plt.contourf(X,Y,Z, 20, cmap=plt.get_cmap('gist_earth'));
plt.title('Barthymetry')
plt.colorbar();

# Create raster tuple
x = X[:,0]
y = Y[0,:]
Zr = np.flipud(Z.T)

raster = x,y,Zr

### Load Incoming Wave 

We will apply an incoming wave on the left boundary. So first we load the data from the `boundary_filename` file. 

From the data we form an interpolation functon called  `wave_function`, which will be used to specify the boundary condition on the left. 

And we also plot the function.  The units of the data in the file are metres, and the scale of the experimental setup is 1 in 400. 

In [ ]:
bdry = np.loadtxt(boundary_filename, skiprows=1)

bdry_t = bdry[:,0]
bdry_v = bdry[:,1]


import scipy
wave_function = scipy.interpolate.interp1d(bdry_t, bdry_v, kind='zero', fill_value='extrapolate')

t = np.linspace(0.0,25.0, 100)

plt.plot(t,wave_function(t)*400);
plt.xlabel('Seconds')
plt.ylabel('Metres')
plt.title('Incoming Wave');

## Setup Domain

We define the `domain` for our simulation. This object encapsulates the mesh for our problem, which is defined by setting up a bounding polygon and associated tagged boundary. We use the `base_resolution` variable to set the maximum area of the triangles of our mesh.  

At the end we use `matplotlib` to visualise the mesh associated with the `domain`.



In [ ]:
base_resolution = 0.01
#base_resolution = 0.0005

# Basic geometry and bounding polygon
xleft   = 0
xright  = 5.448
ybottom = 0
ytop    = 3.402

point_sw = [xleft, ybottom]
point_se = [xright, ybottom]
point_nw = [xleft, ytop]    
point_ne = [xright, ytop]

bounding_polygon = [point_se,
                    point_ne,
                    point_nw,
                    point_sw]

domain = anuga.create_domain_from_regions(bounding_polygon,
                                 boundary_tags={'wall': [0, 1, 3],
                                                'wave': [2]},     
                                 maximum_triangle_area=base_resolution,
                                 use_cache=False,
                                 verbose=False)


domain.set_name('okushiri')  # Name of output sww file 
domain.set_minimum_storable_height(0.001) # Don't store w-z < 0.001m
domain.set_flow_algorithm('DE1')

print ('Number of Elements ', domain.number_of_elements)

domain.set_plotting(min_depth=0.001)  
domain.triplot(linewidth = 0.4);

### Setup Quantities

We use the `raster` created earlier to set the `quantity` called `elevation`.  We also set the  `stage` and the Mannings `friction`. 

We also visualise the `elevation` quantity. 


In [ ]:
domain.set_quantity('elevation',raster=raster, location='centroids')
domain.set_quantity('stage', 0.0)
domain.set_quantity('friction', 0.0025)


domain.tripcolor(facecolors = domain.elev,  
              edgecolors='k', 
              cmap='gist_earth')
plt.colorbar();

### Setup Boundary Conditions

Excuse the verbose boundary type name `Transmissive_n_momentum_zero_t_momentum_set_stage_boundary`, but we use that to set the incoming wave boundary condition. 

On the other boundaries we will have just reflective boundaries.

In [ ]:
Bts = anuga.Transmissive_n_momentum_zero_t_momentum_set_stage_boundary(domain, wave_function)
Br = anuga.Reflective_boundary(domain)

domain.set_boundary({'wave': Bts, 'wall': Br})

### Setup Interogation Variables

We will record the stage at the 3 gauge locations and at the Monai valley. 

In [ ]:
yieldstep = 0.5
finaltime = 22.5

nt = int(finaltime/yieldstep)+1


# area for gulleys
x1 = 4.85
x2 = 5.25
y1 = 2.05
y2 = 1.85

# indices in gulley area
x = domain.centroid_coordinates[:,0]
y = domain.centroid_coordinates[:,1]
v = np.sqrt( (x-x1)**2 + (y-y1)**2 ) + np.sqrt( (x-x2)**2 + (y-y2)**2 ) < 0.5


# Gauge and bounday locations
gauge = [[4.521, 1.196],  [4.521, 1.696],  [4.521, 2.196]] #Ch 5-7-9
bdyloc = [0.00001, 2.5]
g5_id = domain.get_triangle_containing_point(gauge[0])
g7_id = domain.get_triangle_containing_point(gauge[1])
g9_id = domain.get_triangle_containing_point(gauge[2])
bc_id = domain.get_triangle_containing_point(bdyloc)

# Arrays to store data
meanstage = np.nan*np.ones((nt,))
g5 = np.nan*np.ones((nt,))
g7 = np.nan*np.ones((nt,))
g9 = np.nan*np.ones((nt,))
bc = np.nan*np.ones((nt,))

### Evolve

In [ ]:
Stage = domain.quantities['stage'].centroid_values
stage_qoi = Stage[v]

k = 0
for t in domain.evolve(yieldstep=yieldstep, finaltime=finaltime):

    domain.print_timestepping_statistics()
    
    # stage
    stage_qoi = Stage[v]
    meanstage[k] = np.mean(stage_qoi)
    g5[k] = Stage[g5_id]
    g7[k] = Stage[g7_id]
    g9[k] = Stage[g9_id]
    
    k = k+1
     
    #domain.save_depth_frame()


# Read in the png files stored during the evolve loop
#domain.make_depth_animation()

### Animation Using swwfile

Read in the sww file and then iterate through the time slices to produce an animation.

In [ ]:
swwplotter = anuga.SWW_plotter('okushiri.sww', min_depth = 0.001)

n = len(swwplotter.time)

for k in range(n):
  if k%10 == 0:
     print (' ')
  swwplotter.save_stage_frame(frame=k, vmin=-0.02, vmax = 0.1)
  print ('(', swwplotter.time[k], k,')',)
  

print (' ') 
swwplotter.make_stage_animation()

### View Time  Series

In [ ]:
old_figsize = plt.rcParams['figure.figsize']

plt.rcParams['figure.figsize'] = [12, 5]

gauge = np.loadtxt(gauge_filename, skiprows=1)

gauge_t = gauge[:,0]
gauge_5 = gauge[:,1]
gauge_7 = gauge[:,2]
gauge_9 = gauge[:,3]

nt = int(finaltime/yieldstep)+1

import scipy
gauge_5_f = scipy.interpolate.interp1d(gauge_t, gauge_5, kind='zero', fill_value='extrapolate')
gauge_7_f = scipy.interpolate.interp1d(gauge_t, gauge_7, kind='zero', fill_value='extrapolate')
gauge_9_f = scipy.interpolate.interp1d(gauge_t, gauge_9, kind='zero', fill_value='extrapolate')

t = np.linspace(0.0,22.5, 451)

tt= np.linspace(0.0,22.5, nt)

plt.subplot(1,4,1)
plt.plot(t,gauge_5_f(t)*4)
plt.plot(tt,g5*400)
plt.title('Gauge 5')

plt.subplot(1,4,2)
plt.plot(t,gauge_7_f(t)*4)
plt.plot(tt,g7*400)
plt.title('Gauge 7')

plt.subplot(1,4,3)
plt.plot(t,gauge_9_f(t)*4)
plt.plot(tt,g9*400)
plt.title('Gauge 9')
         
plt.subplot(1,4,4)
plt.plot(tt,meanstage*400)
plt.title('Runup');
         

plt.rcParams['figure.figsize'] = old_figsize